In [1]:
#@title Инсталяция библиотек.
#!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu==1.7.4
!pip  install  tiktoken
!pip  install  langchain
!pip  install  openai
!pip  install  chromadb
!pip  install  gspread
!pip  install  oauth2client
!pip  install  nltk
!pip  install  pydantic
!pip  install  faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all t

In [ ]:
!pip list

In [7]:
#@title Импорт библиотек.

#from langchain.llms import OpenAI
#from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate

import gdown
import requests
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken
from openai import OpenAI

In [9]:
#@title Подключение к OpenAI API.
# # Получение ключа API от пользователя и установка его как переменной окружения
# openai_key = getpass.getpass("OpenAI API Key:")
# os.environ["OPENAI_API_KEY"] = openai_key
# #openai.api_key = openai_key

# Установка OpenAI API key
openai.api_key = getpass.getpass("Введите OpenAi API key:")
os.environ["OPENAI_API_KEY"] = openai.api_key


Введите OpenAi API key:··········


In [62]:
#@title Вспомогательные функции

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Возвращает количество токенов в строке"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def split_text(text, max_count, count_type, verbose=0):
    """ Функция разбивает текст на чанки. """

    # Функция для подсчета количества слов в фрагменте.
    def num_words(fragment):
        return  len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте.
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разбивка на чанки происходит в два этапа.
    # На Первом этапе в формате Markdown делится на чанки по по делителям заголовков и подзаголовков.
    # На Втором  этапе полученные чанки делятся при необходимости на более мелкие.

    # Первый этап.
    # Шаблон для MarkdownHeaderTextSplitter по которому будет делится переданный текст в формате Markdown.
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    # Создаем экземпляр спилиттера.
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    # Получаем список чанков.
    fragments = markdown_splitter.split_text(text)

    # Второй этап.
    # Выбор функции подсчета длины в зависимости от типа подсчета.
    length_function = num_words if count_type == "words" else num_tokens
    # Создание объекта разделителя текста.
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)
    # Список для хранения фрагментов текста.
    source_chunks = splitter.split_documents(fragments)

    # Обработка каждого фрагмента текста.
    if verbose:
      for chank in source_chunks:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose.
            count = length_function(chank.page_content)
            answer = f"{count_type} in text fragment = {count}\n{'-' * 5}\n{chank}\n{'=' * 20}"
            print(insert_newlines(answer))

    # Возвращение списка фрагментов текста.
    return source_chunks


def create_embedding(data, max_count, count_type, verbose=0):
    source_chunks = []
    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=verbose)

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type, verbose=0) -> str:
    """Download the document as plain text."""
    try:
        response = requests.get(url) # Получение документа по url.
        response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
        # Создание векторной База знаний.
        search_index = create_embedding(
                          response.text,
                          max_count=max_count,
                          count_type=count_type,
                          verbose=verbose)
        return search_index
    except Exception as e:
        print(e)

def load_file(url: str) -> str:
    try:
        response = requests.get(url) # Получение документа по url.
        response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
        return response.text
    except Exception as e:
        print(e)

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 120) -> str:
    """ Функция форматирует переданный текст по длине
    для лучшего восприятия на экране."""
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 1, verbose = 0, top_similar_documents = 5):
    """ Основная функция которая формирует запрос и получает ответ от OpenAI по заданному вопросу
    на основе векторной Базы знаний. """

    # Выборка чанков по схожести с вопросом.
    docs = search_index.similarity_search(topic, k=top_similar_documents)

    # Формирование контекста на основе выбранных чанков.
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    if (verbose):
        print('\n ===========================================: ')
        print('message_content :\n ======================================== \n', message_content)
    # Формирование запроса к OpenAI.
    messages = [
        {"role": "system", "content": system + f"{message_content}"},
        {"role": "user", "content": topic}
    ]
    # Отправка запроса к Open AI.
    client = OpenAI(
        api_key=openai.api_key
    )
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
    )
    # Вывод на экран подробностей при необходимости.
    if (verbose):
        print('\n ===========================================: ')
        print(f'{completion.usage.prompt_tokens} токенов использовано на вопрос.')
        print('\n ===========================================: ')
        print(f'{completion.usage.total_tokens} токенов использовано всего (вопрос-ответ).')
        print('\n ===========================================: ')
        print('ЦЕНА запроса с ответом :', 0.002*(completion.usage.total_tokens/1000), ' $')
        print('\n ===========================================: ')
    # Ответ OpenAI.
    answer = 'ОТВЕТ : \n' + insert_newlines(completion.choices[0].message.content)
    return answer


In [11]:
#@title Данные для формирования векторной Базы знаний.
count_type = "tokens"       # "words", "token". Тип разбивки на чанки.
max_count = 2000            # Максимальное число слов или токенов.
verbose_bd = 1              # Показывать полученные чанки.
federallab_bd = "https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Datadase/LabSudDB_v1.md" # Сылка на Базу знаний на Github.

# Разбивка Базы знаний идет в два этапа:
# 1. На первом этапе разбивка идет сплитером - MarkdownHeaderTextSplitter. - https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/markdown_header_metadata
#    Текст разбивается на чанки по заголовкам (#, ##, ##) с указанием в метаданных всей иерархии заголовков для этого чанка. Размер этого чанка равен всему объему текста между заголовками.
# 2. На втором этапе чанки после первого этапа разбиваются на чанки сплитером - RecursiveCharacterTextSplitter. - https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
#    Если нет необходимости во втором этапе и вся работа была проведена при формировании Базы знаний по ограничению чанков, то нужно поставить сначение в параметре 'max_count' заведомо большим.

In [ ]:
#@title Формирование векторной Базы знаний.

federallab_bd_index = load_search_indexes(federallab_bd, max_count=max_count, count_type=count_type, verbose=verbose_bd)

In [ ]:
#@title Формулировка промта для нейро-консультанта.

federallab_chat_promt1 = '''Ты консультант компании судебной экспертизы.
У тебя есть подробная информация по услугам и ценам.
Тебе задает вопрос клиент, дай ему информацию, опираясь на предоставленные материалы.
Отвечай максимально точно и используй только информацию из документов, не добавляй ничего своего.
Неупоминай этот документ в ответе.
Документ с информацией для ответа соискателю: '''

federallab_chat_promt2 = load_file("https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Dokumov/%D0%A2%D1%8B%20%D1%81%D0%B0%D0%BC%D1%8B%D0%B9%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B5%D1%82%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9%20%D0%BD%D0%B5%D0%B9%D1%80%D0%BE-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C.txt")
federallab_chat_promt3 = load_file("https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Galina/FLSE_promt")


In [68]:
#@title Вопрос нейро-консультанту:

question = 'Прорводите транспортно-трасологическую экспертизу'

In [73]:
#@title Отправка запроса нейро-консультанту.

top_similar_documents = 3   # Количество полученных релевантных чанков после запроса.
verbose = 1 # Показывать отобранные чанки.
temp = 0    # Вариативность ответа.



In [74]:
answer2 = answer_index(
    system = federallab_chat_promt2,
    topic = question,
    search_index = federallab_bd_index,
    temp = temp,
    verbose = verbose,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Экспертиза обстоятельств ДТП и следов ТС.
Независимая автотехническая экспертиза  
Предметом исследования при проведении экспертизы обстоятельств ДТП являются дорожно-транспортные ситуации (ДТС), параметры движения ТС, иных объектов и пешеходов в процессе ДТП, а также действия и возможности водителей. При транспортно-трасологической экспертизе исследованию подлежат следы различных видов транспорта (от колес, гусениц, выступающих частей ТС, частиц ТС) и механизмы их образования.  
Объектами экспертизы могут быть: дорожное покрытие, транспортные средства, одежда и тело человека, дорожные ограждения и знаки, придорожные постройки.  
Примерные вопросы на разрешение эксперта-трасолога:  
Определить механизм столкновения транспортных средств, представленных на транспортно-трасологическую экспертизу (удар, скольжение, волочение и пр.)?
Определить, имело ли место столкновение представл

In [75]:
answer3 = answer_index(
    system = federallab_chat_promt3,
    topic = question,
    search_index = federallab_bd_index,
    temp = temp,
    verbose = verbose,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Экспертиза обстоятельств ДТП и следов ТС.
Независимая автотехническая экспертиза  
Предметом исследования при проведении экспертизы обстоятельств ДТП являются дорожно-транспортные ситуации (ДТС), параметры движения ТС, иных объектов и пешеходов в процессе ДТП, а также действия и возможности водителей. При транспортно-трасологической экспертизе исследованию подлежат следы различных видов транспорта (от колес, гусениц, выступающих частей ТС, частиц ТС) и механизмы их образования.  
Объектами экспертизы могут быть: дорожное покрытие, транспортные средства, одежда и тело человека, дорожные ограждения и знаки, придорожные постройки.  
Примерные вопросы на разрешение эксперта-трасолога:  
Определить механизм столкновения транспортных средств, представленных на транспортно-трасологическую экспертизу (удар, скольжение, волочение и пр.)?
Определить, имело ли место столкновение представл

In [76]:
print(answer2)
print()
print(answer3)

ОТВЕТ : 
 Да, мы проводим транспортно-трасологическую экспертизу. Эта услуга включает исследование следов различных видов
 транспорта и механизмы их образования, а также определение различных параметров и обстоятельств ДТП.

ОТВЕТ : 
 Да, наша компания проводит транспортно-трасологическую экспертизу. Эта экспертиза включает в себя исследование
 дорожно-транспортных ситуаций, параметров движения транспортных средств, а также исследование следов различных видов
 транспорта и механизмов их образования. Мы можем помочь вам определить механизм столкновения транспортных средств,
 последовательность возникновения повреждений, угол взаимного расположения ТС, траекторию и характер движения ТС, а
 также другие вопросы, связанные с
